<a href="https://colab.research.google.com/github/veerpatel24/Who-Survived-the-Titanic/blob/main/NN_Model_for_Who_Survived_The_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
!ls

gender_submission.csv  sample_data  submission.csv  test.csv  train.csv


In [67]:
import pandas as pd

In [68]:
train_df = pd.read_csv('/content/train.csv').dropna()
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [69]:
test_df = pd.read_csv('/content/test.csv').dropna()
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S
14,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S
24,916,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",female,48.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C
26,918,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C
28,920,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S


In [70]:
import torch
# !pip install pytorch

In [71]:
class Classifier(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()

    # add layers

    # need number of features
    z1 = 16
    z2 = 8

    self.layer1 = torch.nn.Linear(in_features=input_dim, out_features=z1)
    self.layer2 = torch.nn.Linear(in_features=z1, out_features=z2)
    self.layer3 = torch.nn.Linear(in_features=z2, out_features=output_dim)

    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()

    # activation functions


  def forward(self, x):

    out1 = self.layer1(x)

    out1 = self.relu(out1)

    out2 = self.layer2(out1)

    out2 = self.relu(out2)

    out3 = self.layer3(out2)

    out3 = self.sigmoid(out3)

    return out3

In [72]:
train_df.columns


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [73]:
# format data

## print out the features


## cut out the shitty ones

['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin', 'Embarked']

features_list = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']

X_train_df = train_df[features_list]
y_train_df = train_df[['Survived']]
X_test_df = test_df[features_list]
## separate into X and y

In [74]:
X_train_df_encoded = pd.get_dummies(X_train_df, columns=['Sex'], dtype=int)
X_test_df_encoded = pd.get_dummies(X_test_df, columns=['Sex'], dtype=int)

In [75]:
X_train_df_encoded.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
1,1,38.0,1,0,71.2833,1,0
3,1,35.0,1,0,53.1000,1,0
6,1,54.0,0,0,51.8625,0,1
10,3,4.0,1,1,16.7000,1,0
11,1,58.0,0,0,26.5500,1,0


In [76]:
import numpy as np

X_train_np = X_train_df_encoded.to_numpy(dtype=np.float32)
y_train_np = y_train_df.to_numpy()
X_test_np = X_test_df_encoded.to_numpy(dtype=np.float32)

In [77]:
X_train = torch.from_numpy(X_train_np)
y_train = torch.from_numpy(y_train_np)
X_test = torch.from_numpy(X_test_np)

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42,
                                                  shuffle=True)


In [79]:
X_train.shape

torch.Size([155, 7])

In [80]:
in_dim = 7
out_dim = 1

model = Classifier(in_dim, out_dim)

In [81]:
y_train_pred = model(X_train)

In [82]:
prev_val_loss = 10000000000000000

# loss function

loss = torch.nn.BCELoss()

lr = 0.001
opt = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
val_losses = []

y_train = y_train.float()
y_val = y_val.float()
for i in range(100):

  # predict train
  y_train_pred = model(X_train)

  train_loss = loss(y_train_pred, y_train)

  # predict val
  with torch.no_grad():
    y_val_pred = model(X_val)

    val_loss = loss(y_val_pred, y_val)

  train_loss.backward()

  opt.step()
  opt.zero_grad()
    # zero out gradients

  print("Epoch", i, "Train loss:", train_loss, "Val loss:", val_loss)


  train_losses.append(train_loss)
  val_losses.append(val_loss)

Epoch 0 Train loss: tensor(1.1291, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(1.0927)
Epoch 1 Train loss: tensor(1.0304, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.9973)
Epoch 2 Train loss: tensor(0.9409, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.9126)
Epoch 3 Train loss: tensor(0.8639, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.8402)
Epoch 4 Train loss: tensor(0.8017, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.7815)
Epoch 5 Train loss: tensor(0.7545, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.7362)
Epoch 6 Train loss: tensor(0.7195, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.7038)
Epoch 7 Train loss: tensor(0.6942, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.6824)
Epoch 8 Train loss: tensor(0.6763, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.6697)
Epoch 9 Train loss: tensor(0.6641, grad_fn=<BinaryCrossEntropyBackward0>) Val loss: tensor(0.6639)
Epoch 10 T

In [83]:
# submitting to kaggle

## predict on X_test

## convert probabilities into binary classes (0/1)

## turn back into a dataframe containing passenger id and binary classes

## dataframe -> csv file

## submit to kaggle

In [84]:
y_train_pred

tensor([[0.5139],
        [0.6166],
        [0.6428],
        [0.8163],
        [0.6146],
        [0.6179],
        [0.7510],
        [0.5476],
        [0.5745],
        [0.5547],
        [0.6464],
        [0.5384],
        [0.4159],
        [0.6358],
        [0.6468],
        [0.7148],
        [0.5725],
        [0.5832],
        [0.5903],
        [0.6936],
        [0.6199],
        [0.6929],
        [0.6234],
        [0.5143],
        [0.6469],
        [0.6990],
        [0.2310],
        [0.7211],
        [0.6764],
        [0.7051],
        [0.6960],
        [0.6368],
        [0.2973],
        [0.5406],
        [0.6542],
        [0.7035],
        [0.7111],
        [0.5502],
        [0.5524],
        [0.5445],
        [0.7129],
        [0.6789],
        [0.5085],
        [0.6463],
        [0.5644],
        [0.6017],
        [0.5497],
        [0.6084],
        [0.5530],
        [0.5547],
        [0.7238],
        [0.4554],
        [0.6463],
        [0.5358],
        [0.7005],
        [0

# Task
Generate a Kaggle submission file named "submission.csv" from the model's predictions on the test set, converting probabilities to binary classes and including 'PassengerId' and 'Survived' columns.

## Convert Probabilities to Binary Classes

### Subtask:
Convert the predicted probabilities from `test_predictions` into binary classes (0 or 1) by applying a threshold (e.g., 0.5). Survived if probability >= 0.5, else not survived.


**Reasoning**:
The first step is to use the trained model to make predictions on the `X_test` data. This will output probabilities for each instance.



In [85]:
with torch.no_grad():
  test_predictions = model(X_test)
print("Generated test predictions.")

Generated test predictions.


## Making Predictions and Evaluating Model Performance

Although predictions for the submission file (`submission.csv`) have already been generated using the `X_test` dataset, we can explicitly demonstrate how to use the trained model to make predictions and evaluate its performance on the `X_val` (validation) set, for which we have true labels (`y_val`). This will provide insight into how well the model generalizes.


### Predict on the Validation Set

First, we'll use the trained model to predict 'Survived' probabilities for the validation set (`X_val`).

In [86]:
model.eval()
with torch.no_grad():
  val_predictions_proba = model(X_val)

print("Predictions generated for the validation set.")


Predictions generated for the validation set.


### Convert Validation Probabilities to Binary Classes

Next, we convert these probabilities into binary (0 or 1) predictions using a threshold (e.g., 0.5), similar to what was done for the test set.

In [87]:
val_binary_predictions = (val_predictions_proba >= 0.5).int()
print("Validation probabilities converted to binary classes.")


Validation probabilities converted to binary classes.


### Evaluate Model Performance

To make the neural network 'testable', we can evaluate its performance using metrics like accuracy on the validation set, where we have the ground truth (`y_val`).

In [88]:
from sklearn.metrics import accuracy_score

# Convert y_val to a NumPy array for comparison
y_val_np = y_val.cpu().numpy()
val_binary_predictions_np = val_binary_predictions.cpu().numpy()

accuracy = accuracy_score(y_val_np, val_binary_predictions_np)
print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6071


This accuracy score gives you an indication of how well your model performs on unseen data (the validation set).

**Reasoning**:
To convert the probabilistic predictions into binary classifications (0 or 1), a threshold of 0.5 will be applied to the `test_predictions` tensor.



In [89]:
binary_predictions = (test_predictions >= 0.5).int()
print("Converted predictions to binary classes.")

Converted predictions to binary classes.


## Create Submission DataFrame

### Subtask:
Create a pandas DataFrame with 'PassengerId' from the original `test_df` and the newly predicted binary 'Survived' classes. Ensure 'PassengerId' is correctly retrieved from `test_df` before it was modified or encoded.


**Reasoning**:
The first step is to extract the 'PassengerId' column from the original `test_df` to ensure the correct IDs are associated with the predictions.



In [90]:
passenger_ids = test_df['PassengerId']
print("Extracted PassengerId from test_df.")

Extracted PassengerId from test_df.


**Reasoning**:
Now, convert the `binary_predictions` PyTorch tensor into a 1D NumPy array of integers suitable for the 'Survived' column in the submission DataFrame.



In [91]:
survived_predictions = binary_predictions.cpu().numpy().flatten()
print("Converted binary predictions to a 1D NumPy array.")

Converted binary predictions to a 1D NumPy array.


**Reasoning**:
With both 'PassengerId' and the binary 'Survived' predictions ready, the next step is to combine them into a new pandas DataFrame, which will serve as the submission file.



In [92]:
submission_df = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': survived_predictions
})
print("Created submission DataFrame.")

Created submission DataFrame.


**Reasoning**:
The next step is to save the created `submission_df` DataFrame to a CSV file named 'submission.csv' without the index, as required for Kaggle submissions.



In [93]:
submission_df.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created.")

Submission file 'submission.csv' created.


## Final Task

### Subtask:
The model has made predictions on the test set, and a submission file has been generated. You can now submit this file to Kaggle.


## Summary:

### Data Analysis Key Findings

*   The trained model successfully generated probabilistic predictions for the test dataset.
*   These probabilistic predictions were converted into binary class labels (0 for 'Not Survived', 1 for 'Survived') by applying a threshold of 0.5.
*   A pandas DataFrame named `submission_df` was successfully created, containing the original `PassengerId` and the new binary `Survived` predictions.
*   The final submission file, `submission.csv`, was generated and saved without the DataFrame index, in the required format for Kaggle.

### Insights or Next Steps

*   The `submission.csv` file is now ready for submission to the Kaggle competition.
*   Consider evaluating the model's performance on the public leaderboard and potentially explore hyperparameter tuning or different probability thresholds to optimize the submission score.
